# Basic DfT REST API

- Inspo: https://www.tomforth.co.uk/toomanybuses/
- Docs: https://data.bus-data.dft.gov.uk/api/buslocation-openapi/#/SIRI-VM%20Data%20feed/get_datafeed

In [ ]:
import json
import os
from pathlib import Path

import httpx
import xmltodict
from dotenv import load_dotenv
from google.transit.gtfs_realtime_pb2 import FeedMessage
from lxml import etree

In [ ]:
load_dotenv()

## Overall settings

In [ ]:
base_url = "https://data.bus-data.dft.gov.uk/api/v1/"
api_key = os.environ["API_KEY"]
adminArea = 340

## Schedules
Not actually useful to do this, easier to just bulk download the GTFS files

In [ ]:
client = httpx.Client(base_url=base_url, params={"api_key": api_key})

In [ ]:
r = client.get(
    "dataset/", params={"adminArea": adminArea, "limit": 500, "search": "Oxford"}
)
r.status_code

In [ ]:
results = r.json()["results"]
len(results)

In [ ]:
r = results[7]
r["id"], r["operatorName"], r["name"], r["lines"], r["url"]

## Location
Rather use the bods-client Python library

In [ ]:
client = httpx.Client(base_url=base_url, params={"api_key": api_key})

In [ ]:
def get_activity_list(line_ref, op_ref) -> list[dict]:
    r = client.get("datafeed/", params={"lineRef": line_ref, "operatorRef": op_ref})
    assert r.status_code == 200
    d = xmltodict.parse(r.text)
    try:
        va = d["Siri"]["ServiceDelivery"]["VehicleMonitoringDelivery"]["VehicleActivity"]
        # print(json.dumps(va, indent=2))
        return list(va)
    except KeyError:
        return []

In [ ]:
lines = (
    get_activity_list("5", "OXBC")
    + get_activity_list("1", "SCOX")
    + get_activity_list("10", "SCOX")
)
len(lines)

## Location GTFS

In [ ]:
client = httpx.Client(base_url=base_url, params={"api_key": api_key})
r = client.get("gtfsrtdatafeed/", params={"routeId": "3815"})
assert r.status_code == 200

In [ ]:
r.headers

In [ ]:
message = FeedMessage()
message.ParseFromString(r.content)

In [ ]:
message.entity[0]

## Load externally downloaded pb2 file

In [ ]:
path = next(Path("../data").glob("*.pb2"))
message = FeedMessage()
message.ParseFromString(path.read_bytes())

In [ ]:
[e.vehicle.trip.route_id for e in message.entity]